## Example Selection

In [ ]:
import pandas as pd
import json

label_column = 'Class'


train_df = pd.read_csv(f'../../Tasks/elf22/train.csv')
train_df['length'] = train_df['text'].apply(lambda x: len(x))

mapping = {1: "troll", 2: "counterspeech"}


train_df[label_column] = train_df[label_column].apply(lambda x: mapping[x])

context = ""
for j in train_df[label_column].unique():
	print(f'Class: {j}')
	sample = train_df[train_df['length'] < 180]
	sample = sample[sample[label_column] == j].sample(5)
	for index, row in sample.iterrows():
		print(f'  {row["text"]}')
		context += f'<text_icl_begin> CONTEXT: {row["context"]}\nCOMMENT: {row["text"]} <text_icl_end>\nLABEL: {j}\n'

#save the context into a json file
context = f"You are an expert in social psychology.\nAlways read the CONTEXT before the COMMENT and decide whether the comment is a TROLL or COUNTERSPEECH. Reply with the single label on its own line — no extra words.\n\n### EXAMPLES\n{context}### END EXAMPLES"
with open('../icl_promtps/elf22.json', 'w') as f:
	json.dump([context], f, indent=4)

Class: counterspeech
  BD normally doesn’t sell through a 3rd party reseller. If BD doesn’t have it available, you can check eBay.
  You can wait years to slowly progress in pain or just pay money regularly to progress regularly
  Massacre missions don’t stack. The kills count one mission at a time  Only way you can really “stack” them is with a wing all sharing wing massacre missions
  CHIA is for you to plot, and then set aside and forget.   Check it once every few days
  Were all in this together  unless you don't wear a mask and you deserve to be beaten up with a hammer at home depot 🙄
Class: troll
  That's makes you a thief.
  Ok.  I will spare you.   But don't ask questions if you don't like the answers.
  Hey OP, I read your comments and I don't think you should have done this regardless. Good luck with your mum.
  Lol I hate you guys
  Its really funny how you people react to an actual unpopular opinion. Tells me everything that needs to be said


In [ ]:
import pandas as pd
import json

label_column = 'Class'


train_df = pd.read_csv(f'../elf22/train.csv')
train_df['length'] = train_df['text'].apply(lambda x: len(x))

mapping = {1: "troll", 2: "counterspeech"}


train_df[label_column] = train_df[label_column].apply(lambda x: mapping[x])

context = ""
for j in train_df[label_column].unique():
	print(f'Class: {j}')
	sample = train_df[train_df['length'] < 180]
	sample = sample[sample[label_column] == j].sample(5)
	for index, row in sample.iterrows():
		print(f'  {row["text"]}')
		context += f'<text_icl_begin> CONTEXT: {row["context"]}\nCOMMENT: {row["text"]} <text_icl_end>\nLABEL: {j}\n'

#save the context into a json file
context = f"You are an expert in social psychology.\nAlways read the CONTEXT before the COMMENT and decide whether the comment is a TROLL or COUNTERSPEECH. Reply with the single label on its own line — no extra words.\n\n### EXAMPLES\n{context}### END EXAMPLES"
with open('icl_promtps/elf22.json', 'w') as f:
	json.dump([context], f, indent=4)

### Pompt GPT-4o model and save results for each fold

In [5]:
import json 

with open('../icl_promtps/elf22.json', 'r') as f:
	data = json.load(f)[0]

In [ ]:
import pandas as pd
import openai, json
import pandas as pd
from tqdm import tqdm
import traceback, pickle

import os, dotenv
import sys
sys.path.append('../..')

dotenv.load_dotenv()

client = openai.OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

def get_response( message ): 
  
    response = client.chat.completions.create(
      model="chatgpt-4o-latest",
      # logprobs = True,
      messages=[
        {"role": "system", "content": "You are an expert in social psychology."},
        {"role": "user", "content": message},
      ],
       max_tokens = 10,

    )
    # print(response.choices[0].message.content)
    return response.choices[0].message.content


def get_inference(message, label_set):
    
	pred = "unknown"
	for i in range(6):
		try:
			z = get_response(message)
			if z.strip().lower() in label_set:
				pred = z
				break
		except:
			print(traceback.format_exc())
			pass

	return pred

df = pd.read_csv(f'../../Tasks/elf22/test.csv')
df = df.dropna(subset=['text'])
mapping = {1: "troll", 2: "counterspeech"}
label_column = 'Class'

df[label_column] = df[label_column].apply(lambda x: mapping[x])

for i in range(4):

	for index, row in tqdm(df.iterrows(), total=len(df)):

		prompt = f"{data}\n\nCONTEXT: {row['context']}\nTEXT:{row['text']}\nLABEL:"
		
		response = get_inference(prompt, label_set=list(mapping.values()))
		df.at[index, 'predicted_label'] = response

	df.to_csv(f'outputs/elf22/test_{i}.csv', index=False)

In [1]:
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

f1_scores = []
for i in range(4):
	df = pd.read_csv(f'outputs/elf22/test_{i}.csv')
	f1_scores += [f1_score(df['Class'], df['predicted_label'], average='macro')]
	
print('Avg', np.mean(f1_scores))
print('Std', np.std(f1_scores))

Avg 0.6507261239111524
Std 0.005180579221205069
